In [ ]:
import sys
sys.path.append("..")
from utils.dcm_nii_mat_utils import *
from utils.knowledge_filtering import *

In [ ]:
main_path = "../../RSNA-BTC-Datasets"

In [ ]:
dicom_input = f"{main_path}/train"
pat_list = get_patients_list_from_dcm_folder(dicom_input)
print(pat_list)

In [ ]:
dicom_input = f"{main_path}/train"
nifti_output = f"{main_path}/train_nii"
get_nii_from_dcm_folder(dicom_input, nifti_output)

In [ ]:
import csv

out_dict = {}

with open('train_labels.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        out_dict[row["BraTS21ID"]] = row["MGMT_value"]

    #print(out_dict)
    
    with open('train_labels_0.csv', mode='w') as csv_file:
        fieldnames = ['BraTS21ID', 'MGMT_value']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        writer.writeheader()
        for item in out_dict.items():
            if item[1] == "0":
                writer.writerow({"BraTS21ID": item[0], "MGMT_value": item[1]})

    with open('train_labels_1.csv', mode='w') as csv_file:
        fieldnames = ['BraTS21ID', 'MGMT_value']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        writer.writeheader()
        for item in out_dict.items():
            if item[1] == "1":
                writer.writerow({"BraTS21ID": item[0], "MGMT_value": item[1]})

In [ ]:
nifti_output = f"{main_path}/train_nii"
group_nii_files_by_class(nifti_output)

In [ ]:
nifti_output = f"{main_path}/train_nii"
pat_list = get_patients_list_from_nii_folder(nifti_output)

In [ ]:
"""
test_file_error = list()

# Convert DICOM to NIFTI by reporting errors

dicom_input = "/home/uc/rsna-miccai-brain-tumor-radiogenomic-classification/test"
nifti_output = "/home/uc/RSNA-BTC-Datasets/test_nii"
folder = glob(dicom_input + '/*')

for patient_id in tqdm(folder):
    mri_types = glob(patient_id+'/*')
    for patient in mri_types:
        name = os.path.basename(os.path.dirname(patient))
        #print("NAME: "+name)
        #print("PATIENT: "+patient)
        dir_name = os.path.basename(patient)
        try:
            #dicom_input = dicom2nifti.common.read_dicom_directory(patient)
            #print(dicom_input[0])
            dicom2nifti.dicom_series_to_nifti(patient, os.path.join(nifti_output, dir_name, name + '.nii.gz'))
        except dicom2nifti.exceptions.ConversionValidationError as err:
            print("Error with "+name+" in "+dir_name)
            print(err)
            test_file_error.append(name+" - "+dir_name+" - "+str(err))
            pass 
        #except Exception as err:
            #print("Error with "+name+" in "+dir_name)
            #print(err)
            #file_error.append(name+" - "+dir_name)
            #pass

(test_file_error)
"""

In [ ]:
dicom_input = f"{main_path}/test"
nifti_output = f"{main_path}/test_nii"
get_nii_from_dcm_folder(dicom_input, nifti_output)

In [ ]:
nifti_output = f"{main_path}/test_nii"
group_nii_files_by_unknown_class(nifti_output)

In [ ]:
train_csv = "../train_labels.csv"
test_csv = "../sample_submission.csv"
data = import_train_and_test_from_csv(train_csv, test_csv)

In [ ]:
convert_nii_data_to_mat(data, main_path)

In [ ]:
ext = "mat"

dir_path = f"{main_path}/train_"+ext
test_dir_path = f"{main_path}/test_"+ext
mri_types = ["FLAIR", "T1w"]#, "T1wCE", "T2w"]

dataset_0_mat = Dataset(dir_path, mri_types, list_classes=["0"], transform=None, ext=ext)
logging.info("Train/Val datasets size: {}".format(len(dataset_0_mat)))

dataset_1_mat = Dataset(dir_path, mri_types, list_classes=["1"], transform=None, ext=ext)
logging.info("Train/Val datasets size: {}".format(len(dataset_1_mat)))

dataset_test_mat = Dataset(test_dir_path, mri_types, list_classes=["unknown"], transform=None, ext=ext)
logging.info("Test datasets size: {}".format(len(dataset_test_mat)))

In [ ]:
patients_dict = retrieve_patients_list_from_datasets(dataset_0_mat, dataset_1_mat, dataset_test_mat)

In [ ]:
save_kb_volumes_as_mat(dataset_0_mat, "train", 0, patients_dict["train_0"], ["FLAIR", "T1w"], main_path)
save_kb_volumes_as_mat(dataset_1_mat, "train", 1, patients_dict["train_1"], ["FLAIR", "T1w"], main_path)
save_kb_volumes_as_mat(dataset_test_mat, "test", "unknown", patients_dict["test"], ["FLAIR", "T1w"], main_path)